# Fine Data Manipulation with Nested-Pandas

This tutorial will briefly showcase how one would perform data manipulation operations from `pandas`, like adding columns, replacing values, etc. with `nested-pandas`.

In [ ]:
import nested_pandas as npd
from nested_pandas.datasets import generate_data

# Begin by generating an example dataset
ndf = generate_data(5, 20, seed=1)
ndf

In [ ]:
# Show one of the nested dataframes
ndf.iloc[0].nested

## Nested Column Selection

First, we can directly fetch a column from our nested column (aptly called "nested"). For example, below we can fetch the time column, "t", by specifying `"nested.t"` as the column to retrieve. This returns a "flat" view of the nested `t` column, where all rows from all dataframes are present in one dataframe.

In [ ]:
# Directly Nested Column Selection
ndf["nested.t"]

The advantage of the flat view being that this is easily manipulatable just as any `pandas.Series` object. 

In [ ]:
ndf["nested.t"] + 100

## Adding or Replacing Nested Columns

> *A Note on Performance: These operations involve full reconstruction of the nested columns so expect impacted performance when doing this at scale. It may be appropriate to do these operations within reduce functions directly (e.g. subtracting a value from a column) if performance is key.*

We can use the "base_column.nested_sub_column" syntax to also perform operations that add new columns or replace existing columns for a nested column. For example, we can directly replace the "band" column with a new column that appends an additional string to the values.

In [ ]:
# prepend lsst_ to the band column

ndf["nested.band"] = "lsst_" + ndf["nested.band"]

ndf["nested.band"]

Next, we can create a new column in the "nested" column. For example, we can subtract a value from each time value and return it as a new column.

In [ ]:
# create a new "corrected_t" column in "nested"

ndf["nested.corrected_t"] = ndf["nested.t"] - 5

ndf["nested.corrected_t"]

In [ ]:
# Show the first dataframe again
ndf.iloc[0].nested

## Adding New Nested Structures

Finally, we can also add entirely new nested structures using the above syntax.

In [ ]:
ndf["bands.band_label"] = ndf["nested.band"]
ndf

This is functionally equivalent to using `add_nested`:

In [ ]:
ndf.add_nested(ndf["nested.band"].to_frame(), "bands_from_add_nested")